In [ ]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac

# TensorlyのデフォルトバックエンドをNumPyに設定
tl.set_backend('numpy')

# ランダムなテンソルを生成
shape = (4, 5, 6)  # テンソルのサイズ
rank = 3  # 分解するランク

np.random.seed(42)  # 再現性のためのシード設定
tensor = np.random.random(shape)

# マスクを生成 (0または1の値で、一部の要素をマスク)
mask = np.random.choice([0, 1], size=shape, p=[0.2, 0.8])  # 20%の要素をマスク

# マスクを適用したテンソル
masked_tensor = tensor * mask

# CP分解 (マスクを利用)
weights, factors = parafac(masked_tensor, rank=rank, mask=mask, init='random', n_iter_max=100, tol=1e-6)

# 分解結果を表示
print("Weights:")
print(weights)

print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i+1}:")
    print(factor)

# 分解の再構成 (元のテンソルに近似)
reconstructed_tensor = tl.cp_to_tensor((weights, factors))

# 再構成エラーの計算 (元のテンソルと比較)
error = tl.norm(tensor - reconstructed_tensor) / tl.norm(tensor)
print(f"\nReconstruction Error: {error}")

# マスクされた部分のエラーを確認
masked_error = tl.norm((tensor - reconstructed_tensor) * mask) / tl.norm(masked_tensor)
print(f"Reconstruction Error on Masked Elements: {masked_error}")
